# VacationPy

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key= gkey)

 ## Store Part I results into DataFrame
 - Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../Resources/weather_df.csv')
weather_df

,Unnamed: 0,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Taoudenni,56,ML,15,22.68,-3.98,83.46,15.73
1,1,Port Alfred,8,ZA,67,-33.59,26.89,72.77,22.41
2,2,Hermanus,95,ZA,68,-34.42,19.23,68.00,8.99
3,3,Bredasdorp,35,ZA,46,-34.53,20.04,73.40,14.99
4,4,Ushuaia,75,AR,84,-54.80,-68.30,35.60,6.93
...,...,...,...,...,...,...,...,...,...
557,557,Chapecó,0,BR,68,-27.10,-52.62,68.00,6.93
558,558,Pisco,90,PE,88,-13.70,-76.22,63.00,2.24
559,559,Qaqortoq,33,GL,61,60.72,-46.03,14.00,4.70
560,560,Ketchikan,90,US,100,55.34,-131.65,39.20,3.36


## Humidity Heatmap
- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.
- Create a heat map that displays the humidity for every city from the part I of the homework.

In [3]:
# Grab locations and humidity from weather_df
coord = weather_df[["Latitude", "Longitude"]]
hum = weather_df["Humidity"]


fig = gmaps.figure(zoom_level= 3, center= (35, -20))
heat_layer = gmaps.heatmap_layer(coord, weights= hum, dissipating= False, max_intensity= 100,
                                 point_radius= 1)


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

Narrow down the DataFrame to find your ideal weather condition:
- A max temperature lower than 80 degrees but higher than 70.
- Wind speed less than 10 mph.
- Zero cloudiness.


In [4]:
# Set the criteria for the ideal vacation weather condition

best_conditions_df = pd.DataFrame(weather_df, columns = ["City", 
                                                         "Country", 
                                                         "Latitude", 
                                                         "Longitude", 
                                                         "Max Temperature", 
                                                         "Wind Speed", 
                                                         "Cloudiness"])

good_max_temp = (best_conditions_df["Max Temperature"] <= 80) & (best_conditions_df["Max Temperature"] > 70)
good_wind_spd = best_conditions_df["Wind Speed"] < 10
good_cloudiness = best_conditions_df["Cloudiness"] == 0

hotel_df = best_conditions_df[good_max_temp & good_wind_spd & good_cloudiness]


hotel_df["Hotel Name"] = " "


## Hotel Map
Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.  
- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
 

In [5]:
# Loop through hotel_df with api calls
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": gkey,
        }


        lat = row["Latitude"]
        lng = row["Longitude"]

        params["location"] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
      
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Hotel Name
5,Puerto Escondido,MX,15.85,-97.07,77.67,5.46,0,Selina Puerto Escondido
13,Nouadhibou,MR,20.93,-17.03,73.40,4.70,0,Free Zone Hotel Nouadhibou
37,Três de Maio,BR,-27.77,-54.24,70.81,5.88,0,Franken Hotel
77,Teahupoo,PF,-17.83,-149.27,78.80,8.05,0,Vanira Lodge
122,Kiryat Gat,IL,31.61,34.76,79.00,3.18,0,Desert Gat
192,Vao,NC,-22.67,167.48,74.98,9.17,0,OURE TERA beach resort
215,Tura,IN,25.52,90.22,77.36,2.59,0,Hotel Polo Orchid
277,Tautira,PF,-17.73,-149.15,78.80,8.05,0,NaN
369,Florianópolis,BR,-27.61,-48.50,75.00,1.12,0,Majestic Palace Hotel
379,Hervey Bay,AU,-25.30,152.85,78.01,4.00,0,Oaks Hervey Bay Resort and Spa


In [6]:
# Drop the ones with NaN in Hotel Name column
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NaN"]
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Hotel Name
5,Puerto Escondido,MX,15.85,-97.07,77.67,5.46,0,Selina Puerto Escondido
13,Nouadhibou,MR,20.93,-17.03,73.40,4.70,0,Free Zone Hotel Nouadhibou
37,Três de Maio,BR,-27.77,-54.24,70.81,5.88,0,Franken Hotel
77,Teahupoo,PF,-17.83,-149.27,78.80,8.05,0,Vanira Lodge
122,Kiryat Gat,IL,31.61,34.76,79.00,3.18,0,Desert Gat
192,Vao,NC,-22.67,167.48,74.98,9.17,0,OURE TERA beach resort
215,Tura,IN,25.52,90.22,77.36,2.59,0,Hotel Polo Orchid
369,Florianópolis,BR,-27.61,-48.50,75.00,1.12,0,Majestic Palace Hotel
379,Hervey Bay,AU,-25.30,152.85,78.01,4.00,0,Oaks Hervey Bay Resort and Spa
382,Nicoya,CR,10.15,-85.45,72.43,2.06,0,Hotel Doña Marta


- Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [7]:
# Grabbing from stater_code
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:

# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))